# Cointegration. Simple Strategies

In [1]:
import numpy as np
import pandas as pd

import scipy as sp
import scipy.stats as sps
import scipy.optimize as spop

import matplotlib.pyplot as plt

In [10]:
bk = pd.read_csv('../data/BK_2002-01-01_2022-12-31_1d.csv', index_col=0, parse_dates=True)
bac = pd.read_csv('../data/BAC_2002-01-01_2022-12-31_1d.csv', index_col=0, parse_dates=True)

In [12]:
bk = bk['Close']
bac = bac['Close']

In [52]:
# stock = []
# naive1 = []
# naive2 = []
# opt1 = []
# opt2 = []

In [60]:
df = pd.DataFrame({'bk': bk, 'bac': bac})
df['bk_returns'] = np.log(df['bk'].div(df['bk'].shift(1)))
df['bac_returns'] = np.log(df['bac'].div(df['bac'].shift(1)))
df = df.iloc[1:]
df

,bk,bac,bk_returns,bac_returns
Date,,,,
2002-01-02,26.646786,18.864384,0.004647,0.000159
2002-01-03,27.238337,18.843409,0.021957,-0.001113
2002-01-04,27.699894,19.137033,0.016803,0.015462
2002-01-07,28.571007,18.909319,0.030964,-0.011971
2002-01-08,28.167946,18.606695,-0.014208,-0.016133
...,...,...,...,...
2022-12-23,44.312298,32.260765,0.003815,0.002467
2022-12-27,44.481010,32.320374,0.003800,0.001846
2022-12-28,44.292450,32.558830,-0.004248,0.007351


In [61]:
slope, intercept, rvalue, pvalue, stderr = sps.linregress(df['bac'], df['bk'])
alpha, beta = intercept, slope

In [62]:
df['something i will rename later'] = df['bk'] - (df['bac'] * beta + alpha)

In [63]:
df

,bk,bac,bk_returns,bac_returns,something i will rename later
Date,,,,,
2002-01-02,26.646786,18.864384,0.004647,0.000159,-1.590183
2002-01-03,27.238337,18.843409,0.021957,-0.001113,-0.987491
2002-01-04,27.699894,19.137033,0.016803,0.015462,-0.681895
2002-01-07,28.571007,18.909319,0.030964,-0.011971,0.310170
2002-01-08,28.167946,18.606695,-0.014208,-0.016133,0.067850
...,...,...,...,...,...
2022-12-23,44.312298,32.260765,0.003815,0.002467,8.959743
2022-12-27,44.481010,32.320374,0.003800,0.001846,9.096795
2022-12-28,44.292450,32.558830,-0.004248,0.007351,8.781576


In [48]:

# coint = pd.DataFrame()
# coint[0] = bk
# coint[1] = bac

# coint[2] = coint[0]
# coint[3] = coint[1]

# returns
# coint[2] = np.append(np.array(coint[0][1:])/np.array(coint[0][:-1]) - 1, 0)
# coint[3] = np.append(np.array(coint[1][1:])/np.array(coint[1][:-1]) - 1, 0)

# slope, intercept, rvalue, pvalue, stderr = sps.linregress(coint[1], coint[0])
# alpha, beta = intercept, slope

# coint[4] = coint[0] - (coint[1] * beta + alpha)
slope, intercept, rvalue, pvalue, stderr = sps.linregress(np.array(coint[4][:-1]), np.array(coint[4][1:])
                                                          - np.array(coint[4][:-1]))

# coint[5] = coint[2] * (coint[4]<0) + coint[3]*np.sign(coint[4]>0)
# coint[6] = -coint[2]*np.sign(coint[4]) + coint[3]*np.sign(coint[4])